In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
import torch
import os
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.10.0'

In [3]:
# gpus = tf.config.experimental.list_physical_devices("GPU")
# if gpus:
#     # Restrict TensorFlow to only use the first GPU
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, False)
#             tf.config.experimental.set_virtual_device_configuration(
#                 gpu,
#                 [
#                     tf.config.experimental.VirtualDeviceConfiguration(
#                         memory_limit=4096  # set your limit
#                     )
#                 ],
#             )
#         tf.config.experimental.set_visible_devices(gpus[0], "GPU")
#         logical_gpus = tf.config.experimental.list_logical_devices("GPU")
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

if len(physical_devices) == 0:
    print("No GPU devices found. Using CPU.")
else:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


# Data Pre-Processing

Open **kyu_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`. 

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [5]:
df = open('./Training Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
random.shuffle(games)
print(len(games))

118500


Create a dictionary to convert the coordinates from characters to numbers

In [6]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [7]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [8]:
# # Check how many samples can be obtained
# n_games = 0
# n_moves = 0
# for game in games[:500]:
#     n_games += 1
#     moves_list = game.split(',')
#     for move in moves_list:
#         n_moves += 1
# print(f"Total Games: {n_games}, Total Moves: {n_moves}")

The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [9]:
# x = []
# y = []
# for game in games[:500]:
#     moves_list = game.split(',')
#     for count, move in enumerate(moves_list):
#         x.append(prepare_input(moves_list[:count]))
#         y.append(prepare_label(moves_list[count]))
# x = np.array(x)
# y = np.array(y)

In [10]:
# x.shape

In [11]:
# y.shape

In [12]:
# y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [13]:
# x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

### PyTorch version

In [14]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import numpy as np
# from sklearn.model_selection import train_test_split


# # Assuming you have a PyTorch version that supports torch.device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# df = open('./Training Dataset/kyu_train.csv').read().splitlines()
# games = [i.split(',', 2)[-1] for i in df]
# print(len(games))

# chars = 'abcdefghijklmnopqrs'
# coordinates = {k: v for v, k in enumerate(chars)}
# chartonumbers = {k: v for k, v in enumerate(chars)}

# def prepare_input(moves):
#     x = np.zeros((19,19,4))
#     for move in moves:
#         color = move[0]
#         column = coordinates[move[2]]
#         row = coordinates[move[3]]
#         if color == 'B':
#             x[row,column,0] = 1
#             x[row,column,2] = 1
#         if color == 'W':
#             x[row,column,1] = 1
#             x[row,column,2] = 1
#     if moves:
#         last_move_column = coordinates[moves[-1][2]]
#         last_move_row = coordinates[moves[-1][3]]
#         x[row,column,3] = 1
#     x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
#     return x

# def prepare_label(move):
#     column = coordinates[move[2]]
#     row = coordinates[move[3]]
#     return column*19+row

# def accuracy(predictions, targets):
#     with torch.no_grad():
#         _, predicted = torch.max(predictions, 1)
#         correct = (predicted == targets).sum().item()
#         total = targets.size(0)
#         acc = correct / total
#     return acc

# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.conv1 = nn.Conv2d(4, 32, kernel_size=7, padding='same')
#         self.conv2 = nn.Conv2d(32, 32, kernel_size=7, padding='same')
#         self.conv3 = nn.Conv2d(32, 32, kernel_size=5, padding='same')
#         self.conv4 = nn.Conv2d(32, 32, kernel_size=5, padding='same')
#         self.conv5 = nn.Conv2d(32, 32, kernel_size=3, padding='same')
#         self.conv6 = nn.Conv2d(32, 1, kernel_size=3, padding='same')
#         self.flatten = nn.Flatten()
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
#         x = F.relu(self.conv5(x))
#         x = F.relu(self.conv6(x))
#         x = self.flatten(x)
#         return x

# def create_model():
#     model = SimpleModel()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)
#     criterion = nn.CrossEntropyLoss()
#     return model, optimizer, criterion

# model, optimizer, criterion = create_model()
# print(model)
# model.to(device)

# batch_size = 128
# batch = 1

# for batch_start in range(0, len(games), batch_size):
#     print("epoch", batch)
#     batch_end = batch_start + batch_size
#     batch_games = games[batch_start:batch_end]
#     x = []
#     y = []
#     for game in batch_games:
#         moves_list = game.split(',')
#         for count, move in enumerate(moves_list):
#             x.append(prepare_input(moves_list[:count]))
#             y.append(prepare_label(moves_list[count]))

#     x = torch.tensor(np.array(x), dtype=torch.float32).permute(0, 3, 1, 2).to(device)
#     y = torch.tensor(np.array(y), dtype=torch.long).to(device)

#     x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.10)

#     for epoch in range(1):
#         outputs = model(x_train)
#         loss = criterion(outputs, y_train.view(-1))  # Flatten y_train
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print("val_acc:", accuracy(model(x_val), y_val))
#     print("val_loss:", loss.item())

#     if batch % 30 == 0:
#         torch.save(model.state_dict(), f"./models/kyu_{batch}_{accuracy(model(x_val), y_val):.5f}_{loss.item():.5f}.pth")

#     batch += 1


# Training

### Simple DCNN Model:

In [15]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import BatchNormalization

In [16]:
%load_ext tensorboard
%tensorboard --logdir=path/to/logs

Reusing TensorBoard on port 6006 (pid 15348), started 3:17:07 ago. (Use '!kill 15348' to kill it.)

In [17]:
model_num = None

In [18]:
def create_model():
    inputs = Input(shape=(19, 19, 4))

    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [19]:
def create_model1():
    inputs = Input(shape=(19, 19, 4))
    
    x = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)  # Add BatchNormalization
    x = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)

    policy_head = Dense(361, activation='softmax', name='policy')(x)
    value_head = Dense(1, activation='tanh', name='value')(x)

    model = Model(inputs, [policy_head, value_head])

    opt = Adam(learning_rate=0.001)

    model.compile(optimizer=opt,
                  loss={'policy': 'categorical_crossentropy', 'value': 'mean_squared_error'},
                  metrics={'policy': 'accuracy', 'value': 'mae'})
    
    return model


In [20]:
def create_model2():
    inputs = Input(shape=(19, 19, 4))

    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(256, activation='relu')(outputs)
    outputs = Dropout(0.5)(outputs)
    outputs = Dense(361, activation='softmax')(outputs)  # Adjust the number of units based on your output space
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',  # Change to sparse_categorical_crossentropy
                  metrics=['accuracy'])

    return model

In [21]:
model = create_model2()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 64)        12608     
                                                                 
 batch_normalization (BatchN  (None, 19, 19, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 64)        200768    
                                                                 
 batch_normalization_1 (Batc  (None, 19, 19, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 64)        102464

## data generator

In [22]:
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
from tensorflow.keras.callbacks import TensorBoard
import gc
import math
import time


In [23]:

# Reset keras function from: https://github.com/keras-team/keras/issues/12625
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass
    gc.collect()
    # print("Garbage Collected: " + gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1, visible_device_list="0")
    config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
    set_session(tf.compat.v1.Session(config=config))


In [24]:
# Assuming prepare_input and prepare_label functions are defined elsewhere
def data_generator(games, batch_size):
    for batch_start in range(0, len(games), batch_size):
        batch_end = batch_start + batch_size
        batch_games = games[batch_start:batch_end]
        x = []
        y = []
        for game in batch_games:
            moves_list = game.split(',')
            for count, move in enumerate(moves_list):
                x.append(prepare_input(moves_list[:count]))
                y.append(prepare_label(moves_list[count]))

        x = np.array(x)
        y = np.array(y)

        y_one_hot = tf.one_hot(y, depth=19*19)

        x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

        yield (x_train, y_train, x_val, y_val)


In [25]:

batch_size = 512  # Adjust this if needed
batchs = math.ceil(len(games) / batch_size)

# Set up TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=f"kyu_logs/{time.time()}", histogram_freq=1)

for epoch in range(1, 2):
    print("epoch", epoch)
    batch_count = 1
    
    for x_train, y_train, x_val, y_val in data_generator(games, batch_size):
        print(f"{batch_count}/{batchs}")
        history = model.fit(
            x=x_train, 
            y=y_train,
            batch_size=1024,
            epochs=1,
            validation_data=(x_val, y_val),
            callbacks=[tensorboard_callback]  # Add TensorBoard callback here
        )

        if batch_count % 10 == 0:
            model.save(f"./models/kyu_{batch_count}_{history.history['val_accuracy'][0]:.5f}_{history.history['val_loss'][0]:.5f}.h5")

        batch_count += 1
        reset_keras()


epoch 1
1/232
208/208 [==============================] - 17s 78ms/step - loss: 5.8389 - accuracy: 0.0048 - val_loss: 5.8684 - val_accuracy: 0.0025
2/232
208/208 [==============================] - 17s 80ms/step - loss: 5.0051 - accuracy: 0.0311 - val_loss: 4.5373 - val_accuracy: 0.0497
3/232
207/207 [==============================] - 17s 80ms/step - loss: 4.3549 - accuracy: 0.0587 - val_loss: 4.4454 - val_accuracy: 0.0680
4/232
204/204 [==============================] - 16s 80ms/step - loss: 4.1244 - accuracy: 0.0874 - val_loss: 3.9777 - val_accuracy: 0.0984
5/232
212/212 [==============================] - 17s 80ms/step - loss: 3.9818 - accuracy: 0.1283 - val_loss: 3.9228 - val_accuracy: 0.1659
6/232
208/208 [==============================] - 17s 80ms/step - loss: 3.7630 - accuracy: 0.2204 - val_loss: 3.5124 - val_accuracy: 0.3032
7/232
205/205 [==============================] - 16s 80ms/step - loss: 3.4893 - accuracy: 0.3059 - val_loss: 3.2379 - val_accuracy: 0.3518
8/232
206/206 [====

In [26]:

# batch_size = 512  # Adjust this if needed
# batchs = math.ceil(len(games)/batch_size)

# for epoch in range(1, 2):
#     print("epoch", epoch)
#     batch_count = 1
    
#     for x_train, y_train, x_val, y_val in data_generator(games, batch_size):
#         print(f"{batch_count}/{batchs}")
#         history = model.fit(
#             x=x_train, 
#             y=y_train,
#             batch_size=batch_size,
#             epochs=1,
#             validation_data=(x_val, y_val),
#         )

#         if batch_count % 10 == 0:
#             model.save("./models/kyu_" + str(batch_count) + "_"+ str(history.history['val_accuracy'][0])[:5] + "_" + str(history.history['val_loss'][0])[:5] + ".h5")

#         batch_count += 1
#         reset_keras()



In [27]:
print("val_acc:", history.history['val_accuracy'])
model.save(f'./models/model_kyu2.h5')

val_acc: [0.4835520088672638]


## without data generator

In [28]:
# batch_size = 400
# batch = 1

# for batch_start in range(0, 20000, batch_size):
#     print("epoch", batch)
#     batch_end = batch_start + batch_size
#     batch_games = games[batch_start:batch_end]
#     x = []
#     y = []
#     for game in batch_games:
#         moves_list = game.split(',')
#         for count, move in enumerate(moves_list):
#             x.append(prepare_input(moves_list[:count]))
#             y.append(prepare_label(moves_list[count]))

#     x = np.array(x)
#     y = np.array(y)

#     y_one_hot = tf.one_hot(y, depth=19*19)

#     x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)
    
#     history = model.fit(
#         x = x_train, 
#         y = y_train,
#         batch_size = 128,
#         epochs = 1,
#         validation_data=(x_val, y_val),
#     )
#     print("val_acc:", history.history['val_accuracy'])

#     if batch % 10 == 0:
#         model.save("./models/kyu_" + str(batch) + "_"+ str(history.history['val_accuracy'][0])[:5] + "_" + str(history.history['val_loss'][0])[:5] + ".h5")
    
#     tf.keras.backend.clear_session()
#     del history
#     batch += 1


In [29]:
tf.keras.backend.clear_session()
tf.config.experimental.reset_memory_stats('GPU:0')

In [30]:
model.save('./model_kyu_tutorial.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!